In [195]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
%matplotlib inline
pd.set_option('display.max_colwidth',999)

### Variable List
df --> DataFrame (1778,10) with original data

df1 --> DataFrame (1612,7) with unused_cols dropped, and null values from Formatted Result dropped

unused_cols -->  DataFrame (1778,3) columns Pukie, Notes and Description

no_work--> DataFrame (63,7) no work was performed - Warmsups and Cooldowns, not needed for performance calcs

lifts --> DataFrame (45,7) lifts only, work completed but no time.  These will be used to calculate volume abilities. 



In [199]:
df = pd.read_csv('data.csv')

In [197]:
df_wo = pd.read_csv('df_wo.csv')

In [204]:
unused_cols = df[['Pukie','Notes']]
unused_cols.shape

(1778, 2)

In [205]:
df1 = df.drop(['Pukie','Notes'], axis=1)

In [206]:
df1 = df1.dropna(subset=['Formatted Result'])
df1.shape

(1612, 8)

In [207]:
no_work = df1.loc[(df['Formatted Result'] == 'Completed') & (df['Work performed'] == 0), :]
no_work.shape

(63, 8)

In [208]:
lifts = df1.loc[(df1['Work time'].isnull()) & (df1['Formatted Result'].str.endswith('| Completed') | df1['Formatted Result'].str.startswith('Completed |')), :]

In [209]:
movements = pd.read_csv('movements.csv', index_col='Unnamed: 0')
add_moves = [{'movement':'double-under','frequency':153,'move_class':'Monostructural'},{'movement':'squat','frequency':63,'move_class':'Gymnastics'}]
am = pd.DataFrame(add_moves)
new_move = pd.concat([movements,am], ignore_index=True)


In [211]:
new_move_dict = new_move[['movement','move_class']].to_dict(orient='records')

In [212]:
relate_dict = {}
for row in new_move_dict:
    relate_dict[row['movement']] = row['move_class']

In [213]:
move_dict = movements.movement.to_dict()
res = dict((v,k) for k,v in move_dict.iteritems())

In [221]:
workouts = [x for x in df1.Description]
work_string = ' '.join(workouts)


In [222]:
move_in_work = []
for w in workouts:
    moves = []
    for m in new_move.movement:
        if m in w.lower():
            moves.append(m)
    move_in_work.append(moves)
    
bar = [', '.join(x) for x in move_in_work]


In [223]:
df1['Movements'] = bar

In [224]:
df1['power'] = df['Work performed'] / (df['Work time']/1000)

In [225]:
df1 = df1.loc[(df1['Work performed']>1000) & (df1['Work time'] < 5000000),:].copy()

In [226]:
df1.reset_index(drop=True, inplace=True)

In [228]:
df1

,Date,Workout,Result,Prescribed,Work performed,Work time,Formatted Result,Description,Movements,power
0,2017-04-04,"5 RFT: Power Cleans, Walking Lunges, and Thrusters",3.990000e+05,True,42039,399000.0,6 mins 39 secs,"5 rounds of:\n12 Power Cleans, 65 lbs\n6 Walking Lunges, 65 lbs\n3 Thrusters, 65 lbs","lunge, walking lunge, clean, power clean, thruster",105.360902
1,2017-04-02,20:00 AMRAP:\n5 Pull-ups\n10 Push-ups\n15 Air Squats,1.113300e+01,True,88204,1200000.0,11 rounds + 2 Pull-ups | 332 reps,20:00 AMRAP:\n5 Pull-ups\n10 Push-ups\n15 Air Squats,"air squat, pull-up, push-up, squat",73.503333
2,2017-03-28,21-15-9: Push Press and Box Jumps,4.190000e+05,True,21197,419000.0,6 mins 59 secs,"21-15-9 reps of:\nPush Press, 95 lbs\nBox Jump, 20 in","box jump, press, push press",50.589499
3,2017-03-26,"Intervals : rest 3 mins\nRow, 1 km | 4:11.9\nRow, 1 km | 4:19.3\nRow, 1 km | 4:20.3",7.715000e+05,True,93741,771500.0,12 mins 51.5 secs,"Intervals : rest 3 mins\nRow, 1 km | 4:11.9\nRow, 1 km | 4:19.3\nRow, 1 km | 4:20.3",row,NaN
4,2017-03-24,"10 rounds of:\n9 Thrusters, 65 lbs\n35 Double Unders",9.820000e+05,True,66793,982000.0,16 mins 22 secs,"10 rounds of:\n9 Thrusters, 65 lbs\n35 Double Unders","double under, thruster",NaN
5,2017-03-22,Every 1:30 for 10:30: Front Squat,8.572896e+02,True,5848,630000.0,1890 lbs,Every 1:30 for 10:30:\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs\n2 Front Squats | 135 lbs,"front squat, squat",121.504861
6,2017-03-22,AMRAP 5 mins: Squat Cleans and AbMat Sit-ups,3.810000e+00,True,29387,300000.0,3 rounds + 7 Squat Cleans + 13 AbMat Sit-ups | 104 reps,"5:00 AMRAP:\n7 Squat Cleans, 75 lbs\n21 AbMat Sit-ups","abmat sit-up, sit-up, clean, squat clean, squat",68.017312
7,2017-03-22,AMRAP 5 mins: Hanging Knee Raises and Push Press,3.000000e+00,True,2780,300000.0,3 rounds | 84 reps,"5:00 AMRAP:\n21 Hanging Knee Raises\n7 Push Press, 75 lbs","hanging knee raise, press, push press",9.282540
8,2017-03-22,AMRAP 5 mins: Wall Balls and Pull-ups,1.600000e+00,True,10193,300000.0,1 round + 10 Wall Balls + 3 Pull-ups | 38 reps,"5:00 AMRAP:\n10 Wall Balls, 14 lbs\n15 Pull-ups","pull-up, wall ball",97.956667
9,2017-03-21,AMRAP 10 mins: Lateral Burpee (Over Barbell)s and Power Snatches,2.833000e+00,True,44381,600000.0,2 rounds + 15 Lateral Burpee (Over Barbell)s + 10 Power Snatches | 85 reps,"10:00 AMRAP:\n15 Lateral Burpee (Over Barbell)s\n15 Power Snatches, 75 lbs","burpee, lateral burpee (over barbell), power snatch, snatch",9.266667


In [229]:
def mgw_breakdown(cell):
    mgw = []
    move_list = cell.split(', ')
    for x in move_list:
        if x in relate_dict:
            mgw.append(relate_dict[x][0])
    return ''.join(sorted(set(mgw)))
        

In [230]:
from sklearn.feature_extraction.text import CountVectorizer

In [231]:
X = df1['Movements']

vect = CountVectorizer(vocabulary=res)

X_vect = vect.fit_transform(X)

In [232]:
len(X)

741

In [233]:
X_vect

<741x428 sparse matrix of type '<type 'numpy.int64'>'
	with 887 stored elements in Compressed Sparse Row format>

In [234]:
dtm = pd.DataFrame(X_vect.toarray(), columns = vect.get_feature_names())

In [235]:
dtm

,airdyne bike,airdyne calories,assault bike,assault bike calories,"back stroke, swimming",bear crawl,bike,bike ride (mountain bike),"crawl, swimming",double under,...,weighted push-up,weighted sit-up,weighted step ups holding dumbbell,weighted strict bar dip,weighted strict pull-up,weighted strict ring dip,weighted walking lunge,yoke carry,zercher squat,zercher yoke carry
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [240]:
dtm = pd.concat([df1[['Work performed','Work time','power','Movements','Date']], dtm], axis=1)
dtm.shape

(741, 439)

In [237]:
dtm['mgw_breakdown'] = dtm['Movements'].apply(mgw_breakdown)

In [239]:
dtm.to_csv('model_wo.csv')

Basic Linear Regression with Train_test_split

In [ ]:
feature_cols = [c for c in dtm.columns if c != 'power' and c != 'Movements' and c != 'Date' and c!= 'Work performed']
X = dtm[feature_cols]
y = dtm['Work performed']

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(Xs, y, random_state=2008)

lr = LinearRegression()
lr.fit(X_train,y_train)


from sklearn.model_selection import cross_val_score

print 'cross_val_score: {}'.format(cross_val_score(lr, X_train, y_train, cv =5).mean())

pred = lr.predict(X_test)

plt.scatter(pred, pred - y_test)
plt.hlines(y = 0, xmin= pred.min(), xmax=pred.max())

from sklearn.metrics import r2_score
print 'R2 score: {}'.format(r2_score(y_test, pred))



Ridge/Lasso/ElasticNet with GridSearch and Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV, KFold
import numpy as np

In [ ]:
ss = StandardScaler()
Xs = ss.fit_transform(X)

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=101)
ridge = Ridge()
r_params = {
    'alpha': np.logspace(-1, 5, 10),
    'solver':  ['auto', 'sag'],
    'fit_intercept' : [True,False]
}

In [ ]:
gs = GridSearchCV(ridge, cv=kf, param_grid=r_params, scoring='r2')

In [ ]:
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
lasso = Lasso()
l_params = l_params = {
    'alpha': np.linspace(0,5,20),
    'fit_intercept' : [True,False]
}

In [ ]:
gs = GridSearchCV(lasso, cv=kf, param_grid=l_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
enet = ElasticNet()
en_params = {
    'alpha' : np.linspace(0,5,20),
    'l1_ratio' : np.linspace(0.01,1.0,25)
}

In [ ]:
gs = GridSearchCV(enet, cv=kf, param_grid=en_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
from sklearn.kernel_ridge import KernelRidge

In [ ]:
kr = KernelRidge()
kr_params = {
    'alpha': np.logspace(-1, 5, 10)
}

In [ ]:
gs = GridSearchCV(kr, cv=kf, param_grid=kr_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [ ]:
gb = GradientBoostingRegressor()
rf = RandomForestRegressor()

In [ ]:
rf_params = {
    'n_estimators': range(10,30,5),
    'max_features': ['auto', 'sqrt', 'log2', None]
}

In [ ]:
gs = GridSearchCV(rf, cv=kf, param_grid=rf_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_

In [ ]:
gb_params = {
    'n_estimators': [100],
    'loss': ['ls'],
    'max_depth': range(1,11),
    'min_samples_leaf': [1,3,5],
    'max_features' : ['sqrt', 'auto']}

In [ ]:
gs = GridSearchCV(gb, cv=kf, param_grid=gb_params, scoring='r2')
gs.fit(Xs,y)
print gs.best_score_
print gs.best_params_